In [1]:
%cd ..
%load_ext autoreload
%autoreload 2
# %load_ext nb_black

from pathlib import Path
import sys

sys.path.insert(0, Path().absolute().parent.as_posix())
from loguru import logger
import torch

/Users/ruizhechao/Documents/NNforHJB/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/ruizhechao/Documents/NNforHJB


Load the data that is generated from the open-loop optimization

In [2]:
# load the data
import numpy as np

path = 'rawdata/raw_data/VDP_beta_0.1_grid_30x30.npy'
data = np.load(path)
data.dtype
# logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")


dtype([('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')])

In [3]:
data["x"]

array([[-3.        , -3.        ],
       [-2.79310345, -3.        ],
       [-2.5862069 , -3.        ],
       ...,
       [ 2.5862069 ,  3.        ],
       [ 2.79310345,  3.        ],
       [ 3.        ,  3.        ]])

## SSN(trust-region) method for outer weights ##


In [4]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 0.0)
pruning_threshold = 1e-15

gamma = 5.0
alpha = 1e-5
lr_adam = 1e-5
regularization = (gamma, alpha) 
th = 0.0

In [5]:
from scr.model import model
from scr.greedy_insertion import _sample_uniform_sphere_points
import torch

test = model(
    activation=torch.relu, 
    power=power, 
    regularization=regularization, 
    optimizer='SSN', 
    train_outerweights=True)



2025-12-13 14:35:45.245 | INFO     | scr.model:__init__:78 - Model initialized


In [6]:
# prepare the data
# data is a structured numpy array with fields: 'x', 'dv', 'v'
# convert to the dict format expected by model._prepare_data

data_dict = {
    "x": np.asarray(data["x"], dtype=np.float64),    # shape (N, 2)
    "v": np.asarray(data["v"], dtype=np.float64),    # shape (N,)
    "dv": np.asarray(data["dv"], dtype=np.float64),  # shape (N, 2)
}

data_train, data_valid = test._prepare_data(data_dict)

2025-12-13 14:35:45.259 | INFO     | scr.model:_prepare_data:121 - Training set: 810 samples, Validation set: 90 samples
2025-12-13 14:35:45.260 | INFO     | scr.model:_prepare_data:124 - Data ranges - x: [-3.00, 3.00], v: [0.00, 10.96], dv: [-13.19, 13.13]


In [7]:
import torch
from scr.model import model

W_hidden, b_hidden = _sample_uniform_sphere_points(100)  # numpy arrays

W_hidden_t = torch.as_tensor(W_hidden, dtype=torch.float64)
b_hidden_t = torch.as_tensor(b_hidden, dtype=torch.float64).reshape(-1)  # bias must be (M,)

test.train(
    data_train, 
    data_valid, 
    inner_weights=W_hidden_t, 
    inner_bias=b_hidden_t,
    iterations = 1000,
    display_every=100
    )

2025-12-13 14:35:45.273 | INFO     | scr.model:train:278 - Starting network training session
2025-12-13 14:35:45.854 | INFO     | scr.model:_setup_optimizer:198 - Using SSN optimizer with alpha=1e-05, gamma=5.0, th=0.5
2025-12-13 14:35:45.855 | INFO     | scr.model:train:293 - Training hyperparameters: iterations=1000, batch_size=1620, display_every=100
2025-12-13 14:35:45.855 | INFO     | scr.model:train:294 - Loss weights: value=1.0, gradient=1.0
2025-12-13 14:35:45.879 | DEBUG    | scr.ssn:step:130 - ||DG||_inf: 1.96e+01, cond(DG): 2.14e+10, ||Gq||: 1.47e+02
2025-12-13 14:35:45.880 | DEBUG    | scr.ssn:step:144 - Solution (dq) norm: 2.649752e+05
2025-12-13 14:35:45.880 | DEBUG    | scr.ssn:step:156 - ||dq||: 2.649752e+05, ||unew - params||: 2.649752e+05
2025-12-13 14:35:45.882 | DEBUG    | scr.ssn:step:161 - Full step: loss=3.089069e+01 -> loss_new=1.369186e+07
2025-12-13 14:35:45.882 | DEBUG    | scr.ssn:step:186 - LS iter 0: loss_new=1.369186e+07, target=3.089069e+01
2025-12-13 14

KeyboardInterrupt: 

In [ ]:
test_ls = model(activation=torch.relu, power=power, regularization=regularization, optimizer='SSN', loss_weights=loss_weights, th=th, train_outerweights=True)
test.train(data_train, data_valid, inner_weights=W_hidden, inner_bias=b_hidden)

## Test Cases ##

In [ ]:
import torch
from src.ssn import SSN
from src.ssn_tr import SSN_TR

# Build net once if needed
if test.net is None:
    test._create_network(inner_weights=W_hidden, inner_bias=b_hidden)

# Build optimizer once if needed
if test.optimizer is None:
    test._setup_optimizer()

# Define closure for the current data tensors
train_x_tensor, train_v_tensor, train_dv_tensor = data_train
def closure():
    if isinstance(test.optimizer, (SSN, SSN_TR)):
        with torch.no_grad():
            _, hidden_activations = test.net.forward_with_hidden(train_x_tensor.detach())
        test.optimizer.hidden_activations = hidden_activations.detach()
    total_loss, _, _ = test._compute_loss(train_x_tensor, train_v_tensor, train_dv_tensor)
    return total_loss

In [ ]:
from src.utils import _ddphi

params = torch.cat([p.view(-1) for p in test.optimizer.param_groups[0]["params"]])
loss = closure()

q = test.optimizer._transform_param2q(params, loss)
Gq = test.optimizer._Gradient(q, params, loss)

grads = torch.autograd.grad(loss, test.optimizer.param_groups[0]["params"], create_graph=True, retain_graph=True)
grad_flat = torch.cat([g.view(-1) for g in grads])
D_nonconvex = torch.sign(params) * (_ddphi(torch.abs(params), test.th, test.gamma) - 1)

lhs = Gq
rhs = test.optimizer.c * (q - params) + test.alpha * D_nonconvex + grad_flat
print("||Gq||:", float(torch.norm(lhs)), "||lhs - rhs||:", float(torch.norm(lhs - rhs)))

In [ ]:
from src.utils import _compute_prox

params = torch.cat([p.view(-1) for p in test.optimizer.param_groups[0]["params"]])
loss0 = closure()

mu = test.alpha / test.optimizer.c
q = test.optimizer._transform_param2q(params, loss0)
unew = _compute_prox(q, mu)

vals = []
ts = torch.linspace(0, 1, steps=11)
for t in ts:
    u_t = params*(1 - t) + unew*t
    backup = params.clone()
    test.optimizer._update_parameters(u_t)
    vals.append(float(closure()))
    test.optimizer._update_parameters(backup)

print("loss at t grid from params->prox(q):")
for t, v in zip(ts, vals):
    print(f"t={float(t):.1f} loss={v}")
print("Δloss at t=1:", vals[-1] - loss0)

In [ ]:
from src.utils import _compute_dprox, _compute_prox

# Freeze current state
params = torch.cat([p.view(-1) for p in test.optimizer.param_groups[0]["params"]])
loss = closure()

# Use q = params to avoid algebraic cancellation
q = params.clone().detach()
Gq = test.optimizer._Gradient(q, params, loss)          # now ≈ alpha*D_nonconvex + grad_flat
DG = test.optimizer._Hessian(q, params, loss)
mu = test.alpha / test.optimizer.c
DP = _compute_dprox(q, mu)

# One MPCG step
from src.mpcg import mpcg
I_active = (torch.diagonal(DP) != 0)
kmaxit = max(1, int(2 * I_active.sum().item()))
sigma = test.optimizer.sigma if isinstance(test.optimizer, (SSN_TR,)) else 0.0

dq, flag, pred, relres, iters = mpcg(DG, -Gq, 1e-3, kmaxit, sigma, DP)
print("mpcg: flag", flag, "pred", pred, "relres", relres, "iters", iters, "||dq||", float(torch.norm(dq)))

# Try a backtracking grid along dq: q_new = q + t*dq, u_new = prox(q_new)
ts = [1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125]
best = (None, float('inf'))
for t in ts:
    q_t = q + t * dq
    u_t = _compute_prox(q_t, mu)
    backup = params.clone()
    test.optimizer._update_parameters(u_t)
    loss_t = float(closure())
    test.optimizer._update_parameters(backup)
    print(f"t={t:.5f} loss={loss_t} Δ={loss_t - float(loss)}")
    if loss_t < best[1]:
        best = (t, loss_t)
print("best t:", best[0], "best loss:", best[1], "Δbest:", best[1] - float(loss))

In [ ]:
print("sigma:", float(getattr(test.optimizer, "sigma", 0.0)))
DP = __import__("src.utils", fromlist=["_compute_dprox"])._compute_dprox(
    torch.cat([p.view(-1) for p in test.optimizer.param_groups[0]["params"]]),
    test.alpha / test.optimizer.c
)
print("active (DP>0):", int((torch.diagonal(DP) > 0).sum().item()))

In [ ]:
from src.utils import _compute_dprox, _compute_prox, _ddphi
from src.mpcg import mpcg
from src.ssn import SSN
from src.ssn_tr import SSN_TR

train_x_tensor, train_v_tensor, train_dv_tensor = data_train

def closure():
    if isinstance(test.optimizer, (SSN, SSN_TR)):
        with torch.no_grad():
            _, S = test.net.forward_with_hidden(train_x_tensor.detach())
        test.optimizer.hidden_activations = S.detach()
    total_loss, _, _ = test._compute_loss(train_x_tensor, train_v_tensor, train_dv_tensor)
    return total_loss

# Freeze state
params = torch.cat([p.view(-1) for p in test.optimizer.param_groups[0]["params"]])
loss = closure()

# q = params
q = params.clone().detach()
Gq = test.optimizer._Gradient(q, params, loss)
DG = test.optimizer._Hessian(q, params, loss)
mu = test.alpha / test.optimizer.c
DP = _compute_dprox(q, mu)

I_active = (torch.diagonal(DP) != 0)
kmaxit = max(1, int(2 * I_active.sum().item()))
sigma = getattr(test.optimizer, "sigma", 0.0)

dq, flag, pred, relres, iters = mpcg(DG, -Gq, 1e-3, kmaxit, sigma, DP)
print("mpcg: flag", flag, "pred", pred, "relres", relres, "iters", iters, "||dq||", float(torch.norm(dq)))

# Backtracking along dq
ts = [1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125]
best = (None, float('inf'))
for t in ts:
    q_t = q + t * dq
    u_t = _compute_prox(q_t, mu)
    backup = params.clone()
    test.optimizer._update_parameters(u_t)
    loss_t = float(closure())
    test.optimizer._update_parameters(backup)
    print(f"t={t:.5f} loss={loss_t} Δ={loss_t - float(loss)}")
    if loss_t < best[1]:
        best = (t, loss_t)
print("best t:", best[0], "best loss:", best[1], "Δbest:", best[1] - float(loss))

In [ ]:
from copy import deepcopy

params0 = torch.cat([p.view(-1) for p in test.optimizer.param_groups[0]["params"]])
loss0 = float(closure())
print("start loss:", loss0)

mu = test.alpha / test.optimizer.c
history = [loss0]

for k in range(10):  # few iterations to check trend
    params = torch.cat([p.view(-1) for p in test.optimizer.param_groups[0]["params"]])
    loss = closure()
    q = params.clone().detach()
    Gq = test.optimizer._Gradient(q, params, loss)
    DG = test.optimizer._Hessian(q, params, loss)
    DP = _compute_dprox(q, mu)
    I_active = (torch.diagonal(DP) != 0)
    kmaxit = max(1, int(2 * I_active.sum().item()))
    sigma = getattr(test.optimizer, "sigma", 0.0)

    dq, flag, pred, relres, iters = mpcg(DG, -Gq, 1e-3, kmaxit, sigma, DP)

    # simple backtracking
    ts = [1.0, 0.5, 0.25, 0.125, 0.0625]
    chosen = None
    best_loss = float('inf')
    best_u = None
    for t in ts:
        q_t = q + t * dq
        u_t = _compute_prox(q_t, mu)
        backup = params.clone()
        test.optimizer._update_parameters(u_t)
        loss_t = float(closure())
        test.optimizer._update_parameters(backup)
        if loss_t < best_loss:
            best_loss = loss_t
            chosen = t
            best_u = u_t.clone()

    test.optimizer._update_parameters(best_u)
    history.append(best_loss)
    print(f"iter {k}: flag={flag} t={chosen} loss={best_loss}")

print("loss trend:", history)
# restore original if you don't want to keep the change:
# test.optimizer._update_parameters(params0)

In [ ]:
from src.utils import _ddphi, _compute_prox, _compute_dprox

params = torch.cat([p.view(-1) for p in test.optimizer.param_groups[0]["params"]])
loss = closure()
grads = torch.autograd.grad(loss, test.optimizer.param_groups[0]["params"], create_graph=True, retain_graph=True)
grad_flat = torch.cat([g.view(-1) for g in grads])
D_nonconvex = torch.sign(params) * (_ddphi(torch.abs(params), test.th, test.gamma) - 1)
rterm = test.alpha * D_nonconvex + grad_flat
mu = test.alpha / test.optimizer.c

for beta in [0.0, 0.25, 0.5, 0.75, 1.0]:
    q = params - (beta / test.optimizer.c) * rterm
    Gq = test.optimizer._Gradient(q, params, loss)
    DG = test.optimizer._Hessian(q, params, loss)
    DP = _compute_dprox(q, mu)
    I_active = (torch.diagonal(DP) != 0)
    kmaxit = max(1, int(2 * I_active.sum().item()))
    sigma = getattr(test.optimizer, "sigma", 0.0)
    dq, flag, pred, relres, iters = mpcg(DG, -Gq, 1e-3, kmaxit, sigma, DP)

    # evaluate best t along dq
    ts = [1.0, 0.5, 0.25, 0.125]
    best_loss = float('inf')
    for t in ts:
        q_t = q + t * dq
        u_t = _compute_prox(q_t, mu)
        backup = params.clone()
        test.optimizer._update_parameters(u_t)
        loss_t = float(closure())
        test.optimizer._update_parameters(backup)
        best_loss = min(best_loss, loss_t)

    print(f"beta={beta:.2f} ||Gq||={float(torch.norm(Gq)):.3e} flag={flag} best_loss={best_loss} Δ={best_loss - float(loss):.3e}")